<a href="https://colab.research.google.com/github/YaCpotato/ToyBox/blob/master/XGBoostWithIris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## XGBoost

In [0]:
import numpy as np
import matplotlib as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,classification_report
import time
from sklearn import model_selection
import pandas as pd

## データセットロード

In [12]:
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data)
iris_df = iris_df.rename(columns={
    0: 'sepal_length',
    1: 'sepal_width',
    2: 'petal_length',
    3: 'petal_width'})
iris_df['target'] = iris.target
iris_df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [0]:
train_df, test_df = model_selection.train_test_split(iris_df, test_size=0.3) # 30%をテストに使う
train_df_y = train_df[['target']]
train_df_x = train_df.copy().drop('target', axis=1)
test_df_y = test_df[['target']]
test_df_x = test_df.copy().drop('target', axis=1)

In [0]:
import xgboost as xgb
clf = xgb.XGBClassifier()
clf_cv = model_selection.GridSearchCV(clf, {'max_depth': [2,4,6], 'n_estimators': [50,100,200]}, verbose=1)

In [15]:
%%time
clf_cv.fit(train_df_x, [i[0] for i in train_df_y.values])
clf = xgb.XGBClassifier(**clf_cv.best_params_)
clf.fit(train_df_x, [i[0] for i in train_df_y.values])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


CPU times: user 1.02 s, sys: 47.3 ms, total: 1.06 s
Wall time: 1.07 s


[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:    1.0s finished


In [16]:
print(clf_cv.best_params_, clf_cv.best_score_)

{'max_depth': 2, 'n_estimators': 50} 0.9428571428571428


In [17]:
pred = clf.predict(test_df_x)
print(confusion_matrix([i[0] for i in test_df_y.values], pred))
print(classification_report([i[0] for i in test_df_y.values], pred))

[[13  0  0]
 [ 0 14  1]
 [ 0  1 16]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.93      0.93      0.93        15
           2       0.94      0.94      0.94        17

    accuracy                           0.96        45
   macro avg       0.96      0.96      0.96        45
weighted avg       0.96      0.96      0.96        45

